In [1]:
# python train.py \
# --config configs/pose3d/MB_train_h36m.yaml \
# --evaluate checkpoint/pose3d/MB_train_h36m/best_epoch.bin         

In [4]:
import getpass
user = getpass.getuser()
motionbert_root = '/home/{}/codes/MotionBERT'.format(user)

In [6]:
import os
import numpy as np
import argparse
import errno
import math
import pickle
import tensorboardX
from tqdm import tqdm
from time import time
import copy
import random
import prettytable

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

os.chdir(motionbert_root)

from lib.utils.tools import *
from lib.utils.learning import *
from lib.utils.utils_data import flip_data
from lib.data.dataset_motion_2d import PoseTrackDataset2D, InstaVDataset2D
from lib.data.dataset_motion_3d import MotionDataset3D
from lib.data.augmentation import Augmenter2D
from lib.data.datareader_h36m import DataReaderH36M  
from lib.model.loss import *

from train import set_random_seed, save_checkpoint

In [119]:
model_name = 'MB_ft_h36m'
#model_name = 'MB_train_h36m'

In [120]:
import easydict

opts = easydict.EasyDict({
    "config": "configs/pose3d/{}.yaml".format(model_name),
    "checkpoint": 'checkpoint',
    "pretrained": 'checkpoint',
    "resume": '',
    "evaluate": 'checkpoint/pose3d/{}/best_epoch.bin'.format(model_name),
    "selection": 'best_epoch.bin',
    "seed": 0,
    })
set_random_seed(opts.seed)
args = get_config(opts.config)

In [121]:
try:
    os.makedirs(opts.checkpoint)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise RuntimeError('Unable to create checkpoint directory:', opts.checkpoint)
train_writer = tensorboardX.SummaryWriter(os.path.join(opts.checkpoint, "logs"))

In [122]:
args.batch_size

16

In [123]:
args.subset_list

['H36M-SH']

In [124]:
print('Loading dataset...')
trainloader_params = {
        'batch_size': args.batch_size,
        'shuffle': True,
        'num_workers': 12,
        'pin_memory': True,
        'prefetch_factor': 4,
        'persistent_workers': True
}

testloader_params = {
        'batch_size': args.batch_size,
        'shuffle': False,
        'num_workers': 12,
        'pin_memory': True,
        'prefetch_factor': 4,
        'persistent_workers': True
}

train_dataset = MotionDataset3D(args, args.subset_list, 'train')
test_dataset = MotionDataset3D(args, args.subset_list, 'test')
train_loader_3d = DataLoader(train_dataset, **trainloader_params)
test_loader = DataLoader(test_dataset, **testloader_params)

Loading dataset...


In [125]:
datareader = DataReaderH36M(n_frames=args.clip_len, sample_stride=args.sample_stride, data_stride_train=args.data_stride, data_stride_test=args.clip_len, dt_root = 'data/motion3d', dt_file=args.dt_file)

In [126]:
min_loss = 100000
model_backbone = load_backbone(args)
model_params = 0
for parameter in model_backbone.parameters():
    model_params = model_params + parameter.numel()
print('INFO: Trainable parameter count:', model_params)

if torch.cuda.is_available():
    model_backbone = nn.DataParallel(model_backbone)
    model_backbone = model_backbone.cuda()

INFO: Trainable parameter count: 42466317


In [127]:
args.finetune, opts.resume, opts.evaluate

(True, '', 'checkpoint/pose3d/MB_ft_h36m/best_epoch.bin')

In [128]:
chk_filename = opts.evaluate if opts.evaluate else opts.resume
print('Loading checkpoint', chk_filename)
checkpoint = torch.load(chk_filename, map_location=lambda storage, loc: storage)
model_backbone.load_state_dict(checkpoint['model_pos'], strict=True)
model_pos = model_backbone

Loading checkpoint checkpoint/pose3d/MB_ft_h36m/best_epoch.bin


In [129]:
args.partial_train

In [130]:
opts.evaluate

'checkpoint/pose3d/MB_ft_h36m/best_epoch.bin'

#### evaluate

In [131]:
# args, model_pos, test_loader, datareader

In [132]:
args.no_conf, args.flip, args.rootrel, args.gt_2d

(False, True, False, False)

In [142]:
args.flip = True

In [143]:
torch.cuda.is_available()

True

In [144]:
results_all = []
model_pos.eval()            
with torch.no_grad():
    for batch_input, batch_gt in tqdm(test_loader):
        N, T = batch_gt.shape[:2] # B, N
        if torch.cuda.is_available():
            batch_input = batch_input.cuda()
        if args.flip:    
            batch_input_flip = flip_data(batch_input)
            predicted_3d_pos_1 = model_pos(batch_input)
            predicted_3d_pos_flip = model_pos(batch_input_flip)
            predicted_3d_pos_2 = flip_data(predicted_3d_pos_flip)                   # Flip back
            predicted_3d_pos = (predicted_3d_pos_1+predicted_3d_pos_2) / 2
        else:
            predicted_3d_pos = model_pos(batch_input)
        results_all.append(predicted_3d_pos.cpu().numpy())
results_all = np.concatenate(results_all)
results_all = datareader.denormalize(results_all)

100%|██████████| 140/140 [01:30<00:00,  1.55it/s]


In [145]:
np.save('custom_codes/{}_result_denormalized.npy'.format(model_name), results_all)

In [146]:
results_all = np.load('custom_codes/{}_result_denormalized.npy'.format(model_name))

In [147]:
_, split_id_test = datareader.get_split_id()
actions = np.array(datareader.dt_dataset['test']['action'])
factors = np.array(datareader.dt_dataset['test']['2.5d_factor'])
gts = np.array(datareader.dt_dataset['test']['joints_2.5d_image'])
sources = np.array(datareader.dt_dataset['test']['source'])

num_test_frames = len(actions)
frames = np.array(range(num_test_frames))
action_clips = actions[split_id_test]
factor_clips = factors[split_id_test]
source_clips = sources[split_id_test]
frame_clips = frames[split_id_test]
gt_clips = gts[split_id_test]
assert len(results_all)==len(action_clips)

In [148]:
e1_all = np.zeros(num_test_frames)
e2_all = np.zeros(num_test_frames)
oc = np.zeros(num_test_frames)
action_names = sorted(set(datareader.dt_dataset['test']['action']))
block_list = ['s_09_act_05_subact_02', 
                's_09_act_10_subact_02', 
                's_09_act_13_subact_01']

for idx in range(len(action_clips)):
    source = source_clips[idx][0][:-6]
    if source in block_list:
        continue
    frame_list = frame_clips[idx] # numpy.ndarray
    action = action_clips[idx][0]
    factor = factor_clips[idx][:,None,None]
    gt = gt_clips[idx]
    pred = results_all[idx]
    pred *= factor
    
    # Root-relative Errors
    pred = pred - pred[:,0:1,:] # (243, 17, 3)
    gt = gt - gt[:,0:1,:] # (243, 17, 3)
    err1 = mpjpe(pred, gt) # (243,)
    err2 = p_mpjpe(pred, gt) # (243,)
    e1_all[frame_list] += err1 # numpy.ndarray를 인덱스로 사용 가능
    e2_all[frame_list] += err2
    oc[frame_list] += 1 # 프레임별 카운팅

In [149]:
results = {}
results_procrustes = {}

for action in action_names:
    results[action] = []
    results_procrustes[action] = []

for idx in range(num_test_frames):
    if e1_all[idx] > 0:
        err1 = e1_all[idx] / oc[idx]
        err2 = e2_all[idx] / oc[idx]
        action = actions[idx]
        results[action].append(err1)
        results_procrustes[action].append(err2)

In [150]:
final_result = []
final_result_procrustes = []
summary_table = prettytable.PrettyTable()
summary_table.field_names = ['test_name'] + action_names
for action in action_names:
    final_result.append(np.mean(results[action]))
    final_result_procrustes.append(np.mean(results_procrustes[action]))
summary_table.add_row(['P1'] + final_result)
summary_table.add_row(['P2'] + final_result_procrustes)
print(summary_table)
e1 = np.mean(np.array(final_result))
e2 = np.mean(np.array(final_result_procrustes))
print('Protocol #1 Error (MPJPE):', e1, 'mm')
print('Protocol #2 Error (P-MPJPE):', e2, 'mm')
print('----------')

+-----------+--------------------+-------------------+--------------------+--------------------+-------------------+-------------------+--------------------+--------------------+--------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
| test_name |     Direction      |      Discuss      |       Eating       |       Greet        |       Phone       |       Photo       |        Pose        |      Purchase      |      Sitting       |    SittingDown    |       Smoke       |        Wait        |        Walk        |      WalkDog       |      WalkTwo       |
+-----------+--------------------+-------------------+--------------------+--------------------+-------------------+-------------------+--------------------+--------------------+--------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|     P1    | 34.85464262441

#### denormalize

In [73]:
test_data = results_all
n_clips = test_data.shape[0]
test_hw = datareader.get_hw()
data = test_data.reshape([n_clips, -1, 17, 3])

In [74]:
len(data), len(test_hw)

(32, 2228)

In [75]:
for idx, item in enumerate(data):
    res_w, res_h = test_hw[idx]
    data[idx, :, :, :2] = (data[idx, :, :, :2] + np.array([1, res_h / res_w])) * res_w / 2
    data[idx, :, :, 2:] = data[idx, :, :, 2:] * res_w / 2

#### ---

In [91]:
_, split_id_test = datareader.get_split_id()
split_id_test = split_id_test[:len(results_all)]
actions = np.array(datareader.dt_dataset['test']['action'])
factors = np.array(datareader.dt_dataset['test']['2.5d_factor'])
gts = np.array(datareader.dt_dataset['test']['joints_2.5d_image'])
sources = np.array(datareader.dt_dataset['test']['source'])

In [92]:
len(results_all)*243

7776

In [94]:
num_test_frames = len(results_all)*243 # 566920
frames = np.array(range(num_test_frames))
action_clips = actions[split_id_test]
factor_clips = factors[split_id_test]
source_clips = sources[split_id_test]
frame_clips = frames[split_id_test]
gt_clips = gts[split_id_test]

IndexError: index 7776 is out of bounds for axis 0 with size 7776

In [85]:
e1_all = np.zeros(results_all)
e2_all = np.zeros(results_all)
oc = np.zeros(results_all)
results = {}
results_procrustes = {}
action_names = sorted(set(datareader.dt_dataset['test']['action']))
for action in action_names:
    results[action] = []
    results_procrustes[action] = []
block_list = ['s_09_act_05_subact_02', 
                's_09_act_10_subact_02', 
                's_09_act_13_subact_01']

TypeError: only integer scalar arrays can be converted to a scalar index

In [79]:
for idx in range(len(results_all)):
    source = source_clips[idx][0][:-6]
    if source in block_list:
        continue
    frame_list = frame_clips[idx]
    action = action_clips[idx][0]
    factor = factor_clips[idx][:,None,None]
    gt = gt_clips[idx]
    pred = results_all[idx]
    pred *= factor
    
    # Root-relative Errors
    pred = pred - pred[:,0:1,:]
    gt = gt - gt[:,0:1,:]
    err1 = mpjpe(pred, gt)
    err2 = p_mpjpe(pred, gt)
    e1_all[frame_list] += err1
    e2_all[frame_list] += err2
    oc[frame_list] += 1

In [81]:
num_test_frames

566920

In [80]:
for idx in range(num_test_frames):
    if e1_all[idx] > 0:
        err1 = e1_all[idx] / oc[idx]
        err2 = e2_all[idx] / oc[idx]
        action = actions[idx]
        results[action].append(err1)
        results_procrustes[action].append(err2)